In [56]:
import os
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import plotly
import plotly.graph_objects as go
import plotly.express as px
import kaleido ##pip install -U kaleido ##to save a plotly fig
import sys 
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score,cross_val_predict
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import metrics

In [106]:
df1 = pd.read_csv('Stable_Data_CSV.csv')
df2 = pd.read_csv('6wUsingBL.csv')
df_stable = df2[df2.set_index(['participant_id']).index.isin(df1.set_index(['participant_id']).index)]

In [107]:
df_stable_US = df_stable.loc[(df_stable['arm'] == 1)]
df_stable_WN = df_stable.loc[(df_stable['arm'] == 2)]
print(len(df_stable_US))
print(len(df_stable_WN))
df_stable_US=(df_stable_US[['c_3','sl_1','a_2','q_1','tfi_total.1','tfi_total.2','tfi_total.3']])
df_stable_WN=(df_stable_WN[['sl_2','r_1','e_1','tfi_total.1','tfi_total.2','tfi_total.3']])

31
30


In [116]:
# with scaled data

n_epochs=200
n_batch_size=10
n_timesteps=1
lr=0.001
kfold = KFold(n_splits=5)
error_scores_US=[]
error_scores_WN=[]

def data_prep(df):
    sc_x=StandardScaler()
    sc_y=StandardScaler()
    ## Separate dependent and independent features 
    X=df.drop(['tfi_total.2','tfi_total.3'],axis=1).values
    y = df[['tfi_total.2']].values
    #scale x
    x_scaler=sc_x.fit(X)
    X=x_scaler.transform(X)
    #scale y
    y_scaler=sc_y.fit(y)
    y=y_scaler.transform(y)
    # Reshape input to be 3D for LSTM[samples, timesteps, features]
    X = X.reshape((X.shape[0],n_timesteps,X.shape[1]))
    print(X.shape,y.shape)
    input_shape=(n_timesteps,X.shape[2])
    print(input_shape)
    return X,y,input_shape,x_scaler,y_scaler

def create_lstm_model(input_shape):
    tf.keras.backend.clear_session
    input_shape=input_shape
    model = tf.keras.Sequential([
      tf.keras.layers.LSTM(50, activation='relu',kernel_initializer='normal',input_shape=input_shape),
      tf.keras.layers.Dropout(0.2), # 20% of Neuron will get deactivated during training
      tf.keras.layers.Dense(1,kernel_initializer='normal',activation = 'linear')                      
    ])             

    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=lr),metrics=['mae'])
    return model

def cross_val(df,error_scores):
    X,y,input_shape,x_scaler,y_scaler=data_prep(df)
    for train_index, test_index in kfold.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model=create_lstm_model(input_shape)
        model.fit(x_train, y_train, epochs=50, batch_size=10)
        val_mse,val_mae=model.evaluate(x_test,y_test)
        error_scores.append(val_mae)
    return x_scaler,y_scaler,error_scores

US_x_scaler,US_y_scaler,US_error_scores=cross_val(df_stable_US,error_scores_US)
WN_x_scaler,WN_y_scaler,WN_error_scores=cross_val(df_stable_WN,error_scores_WN)
    

(31, 1, 5) (31, 1)
(1, 5)
TRAIN: [ 7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30] TEST: [0 1 2 3 4 5 6]


Epoch 1/50


2022-05-16 14:43:54.402010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 45ms/step - loss: 0.9107 - mae: 0.8576
Epoch 2/50
3/3 [==============================] - 0s 17ms/step - loss: 0.9094 - mae: 0.8568
Epoch 3/50
3/3 [==============================] - 0s 16ms/step - loss: 0.9086 - mae: 0.8565
Epoch 4/50
3/3 [==============================] - 0s 17ms/step - loss: 0.9075 - mae: 0.8562
Epoch 5/50
3/3 [==============================] - 0s 15ms/step - loss: 0.9055 - mae: 0.8554
Epoch 6/50
3/3 [==============================] - 0s 16ms/step - loss: 0.9030 - mae: 0.8546
Epoch 7/50
3/3 [==============================] - 0s 13ms/step - loss: 0.9019 - mae: 0.8534
Epoch 8/50
3/3 [==============================] - 0s 14ms/step - loss: 0.9014 - mae: 0.8534
Epoch 9/50
3/3 [==============================] - 0s 14ms/step - loss: 0.9023 - mae: 0.8538
Epoch 10/50
3/3 [==============================] - 0s 13ms/step - loss: 0.8999 - mae: 0.8525
Epoch 11/50
3/3 [==============================] - 0s 13ms/step - loss: 0.8949 - mae: 0.85

2022-05-16 14:43:56.860953: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-05-16 14:44:19.181511: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50


2022-05-16 14:44:19.818549: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 32ms/step - loss: 1.0573 - mae: 0.9301
Epoch 2/50
3/3 [==============================] - 0s 17ms/step - loss: 1.0563 - mae: 0.9298
Epoch 3/50
3/3 [==============================] - 0s 18ms/step - loss: 1.0559 - mae: 0.9294
Epoch 4/50
3/3 [==============================] - 0s 14ms/step - loss: 1.0477 - mae: 0.9260
Epoch 5/50
3/3 [==============================] - 0s 13ms/step - loss: 1.0497 - mae: 0.9269
Epoch 6/50
3/3 [==============================] - 0s 13ms/step - loss: 1.0436 - mae: 0.9243
Epoch 7/50
3/3 [==============================] - 0s 12ms/step - loss: 1.0421 - mae: 0.9235
Epoch 8/50
3/3 [==============================] - 0s 13ms/step - loss: 1.0452 - mae: 0.9242
Epoch 9/50
3/3 [==============================] - 0s 12ms/step - loss: 1.0434 - mae: 0.9230
Epoch 10/50
3/3 [==============================] - 0s 12ms/step - loss: 1.0356 - mae: 0.9202
Epoch 11/50
3/3 [==============================] - 0s 12ms/step - loss: 1.0332 - mae: 0.92

2022-05-16 14:44:22.007038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [117]:
print(US_error_scores)
print(WN_error_scores)

[0.9196298718452454, 0.9018526077270508, 1.0266377925872803, 0.8595988154411316, 0.8866043090820312]
[0.9157657623291016, 0.5782689452171326, 0.5972484946250916, 1.3093634843826294, 0.5135211944580078]


In [118]:
US_error=np.mean(US_error_scores)
print(US_error)
WN_error=np.mean(WN_error_scores)
print(WN_error)

0.9188646793365478
0.7828335762023926


In [109]:
#without scaling

n_epochs=200
n_batch_size=10
n_timesteps=1
lr=0.001
kfold = KFold(n_splits=5)
error_scores_US=[]
error_scores_WN=[]

def data_prep(df):
    ## Separate dependent and independent features 
    X=df.drop(['tfi_total.2','tfi_total.3'],axis=1).values
    y = df[['tfi_total.2']].values
    # Reshape input to be 3D for LSTM[samples, timesteps, features]
    X = X.reshape((X.shape[0],n_timesteps,X.shape[1]))
    print(X.shape,y.shape)
    input_shape=(n_timesteps,X.shape[2])
    print(input_shape)
    return X,y,input_shape
def create_lstm_model(input_shape):
    tf.keras.backend.clear_session
    input_shape=input_shape
    model = tf.keras.Sequential([
      tf.keras.layers.LSTM(50, activation='relu',kernel_initializer='normal',input_shape=input_shape),
      tf.keras.layers.Dropout(0.2), # 20% of Neuron will get deactivated during training
      tf.keras.layers.Dense(1,kernel_initializer='normal',activation = 'linear')                      
    ])             

    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=lr),metrics=['mae'])
    return model

def cross_val(df,error_scores):
    X,y,input_shape=data_prep(df)
    for train_index, test_index in kfold.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model=create_lstm_model(input_shape)
        model.fit(x_train, y_train, epochs=50, batch_size=10)
        val_mse,val_mae=model.evaluate(x_test,y_test)
        error_scores.append(val_mae)
    return error_scores

US_error_scores=cross_val(df_stable_US,error_scores_US)
WN_error_scores=cross_val(df_stable_WN,error_scores_WN)
    

(31, 1, 5) (31, 1)
(1, 5)
TRAIN: [ 7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30] TEST: [0 1 2 3 4 5 6]


Epoch 1/50


2022-05-16 12:56:57.065407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 38ms/step - loss: 1851.5193 - mae: 39.1708
Epoch 2/50
3/3 [==============================] - 0s 17ms/step - loss: 1839.1152 - mae: 39.0071
Epoch 3/50
3/3 [==============================] - 0s 16ms/step - loss: 1823.1248 - mae: 38.8251
Epoch 4/50
3/3 [==============================] - 0s 13ms/step - loss: 1806.1133 - mae: 38.6123
Epoch 5/50
3/3 [==============================] - 0s 15ms/step - loss: 1789.2156 - mae: 38.3675
Epoch 6/50
3/3 [==============================] - 0s 14ms/step - loss: 1765.6654 - mae: 38.0904
Epoch 7/50
3/3 [==============================] - 0s 12ms/step - loss: 1740.8740 - mae: 37.7775
Epoch 8/50
3/3 [==============================] - 0s 12ms/step - loss: 1716.3177 - mae: 37.4123
Epoch 9/50
3/3 [==============================] - 0s 13ms/step - loss: 1678.3407 - mae: 36.9575
Epoch 10/50
3/3 [==============================] - 0s 13ms/step - loss: 1641.7290 - mae: 36.4516
Epoch 11/50
3/3 [==============================] -

2022-05-16 12:56:59.437368: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2022-05-16 12:57:22.340659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50


2022-05-16 12:57:22.869259: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 32ms/step - loss: 2320.5029 - mae: 43.4931
Epoch 2/50
3/3 [==============================] - 0s 15ms/step - loss: 2301.8914 - mae: 43.3329
Epoch 3/50
3/3 [==============================] - 0s 16ms/step - loss: 2290.5596 - mae: 43.1380
Epoch 4/50
3/3 [==============================] - 0s 14ms/step - loss: 2273.6511 - mae: 42.9577
Epoch 5/50
3/3 [==============================] - 0s 19ms/step - loss: 2253.5349 - mae: 42.7345
Epoch 6/50
3/3 [==============================] - 0s 17ms/step - loss: 2235.9807 - mae: 42.5591
Epoch 7/50
3/3 [==============================] - 0s 13ms/step - loss: 2214.3950 - mae: 42.2638
Epoch 8/50
3/3 [==============================] - 0s 12ms/step - loss: 2182.1558 - mae: 41.9380
Epoch 9/50
3/3 [==============================] - 0s 15ms/step - loss: 2161.8503 - mae: 41.6806
Epoch 10/50
3/3 [==============================] - 0s 11ms/step - loss: 2124.8367 - mae: 41.2082
Epoch 11/50
3/3 [==============================] -

2022-05-16 12:57:25.170959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [114]:
print(US_error_scores)
print(WN_error_scores)

[15.711621284484863, 15.141166687011719, 11.872230529785156, 17.281280517578125, 15.604510307312012]
[19.223968505859375, 10.566120147705078, 14.168830871582031, 22.3701229095459, 11.024335861206055]


In [115]:
US_error=np.mean(US_error_scores)
print(US_error)
WN_error=np.mean(WN_error_scores)
print(WN_error)

15.122161865234375
15.470675659179687
